In [2]:
!pip install nltk

In [1]:
!pip install creme


     ---------------------------------------- 0.0/524.7 kB ? eta -:--:--
      --------------------------------------- 10.2/524.7 kB ? eta -:--:--
     -- ---------------------------------- 41.0/524.7 kB 487.6 kB/s eta 0:00:01
     ----------------- -------------------- 245.8/524.7 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 524.7/524.7 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for creme: filename=creme-0.6.1-cp311-cp311-win_amd64.whl size=806368 sha256=a41619c2f09cc44e65b10667e8c7165dfd7b583b718e93e80069db7d5e8d2f58
  Stored in directory: c:\users\pulki\appdata\local\pip\cache\wheels\0d\8d\e2\c284468e530880caaf1b1896798299b066889164f8ff4fcb90
  Created wheel for mmh3: filename=mmh3-2.5.1-cp311-cp311-win_amd64.whl size=11967 sha256=373e96d6f70661743070db9fe5cdacb6

In [1]:
from sklearn.model_selection import train_test_split
from creme import metrics
import creme
from creme import naive_bayes
from creme import feature_extraction
from creme import compose
import math
import pickle
from nltk.stem.porter import PorterStemmer
import string
from nltk.corpus import stopwords
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
# stopwords.words('english')
# string.punctuation
ps = PorterStemmer()
# ps.stem('worries')
import nltk

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pulki\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pulki\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Load the model from the file
# with open('predict-doc.pkl', 'rb') as f:
#     loaded_model = pickle.load(f)

In [3]:
def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)

    y = []
    for i in text:
        if i.isalnum():
            y.append(i)

    text = y[:]
    y.clear()

    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)

    text = y[:]
    y.clear()

    for i in text:
        y.append(ps.stem(i))

    return " ".join(y)

In [4]:
def preprocesse_df(df):
    df = df.dropna()
    df.duplicated().sum()
    df = df.drop_duplicates(keep='first')
    df['transformed_content'] = df['content'].apply(transform_text)
    processed_df = df[df['content'].str.len() >= 1000]

    return processed_df

In [5]:
import os
import numpy as np

# Define the path to the train folder
train_folder_path = './db/train/'

# Get a list of all filenames in the train folder
filenames = os.listdir(train_folder_path)

# Filter out only the files (excluding directories)
filenames = [filename for filename in filenames if os.path.isfile(
    os.path.join(train_folder_path, filename))]

# Convert the list of filenames to a NumPy array
filenames_array = np.array(filenames)

# Display the filenames array
print(filenames_array)

['Business Documents.csv' 'Creative Documents.csv'
 'Educational Documents.csv' 'Financial Documents.csv'
 'Govt Documents.csv' 'Legal Documents.csv' 'Medical Documents.csv'
 'News Documents.csv' 'Scientific Doc2.csv' 'Scientific Documents.csv'
 'Technical Documents.csv']


In [6]:
df = pd.read_csv('./db/train/'+filenames_array[0])
df

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 98233: invalid start byte

In [7]:
# Load the model from the file
# with open('predict-doc2.pkl', 'rb') as f:
#     loaded_model = pickle.load(f)

model = compose.Pipeline(
    ('tokenize', feature_extraction.BagOfWords(lowercase=False)),
    ('nb', naive_bayes.MultinomialNB(alpha=1))
)

# model = loaded_model

In [21]:
# Initialize lists to store true and predicted labels
true_labels = []
predicted_labels = []

In [22]:
for i,file in enumerate(filenames_array):
#     if i<=8:
#         continue
    print(i, file, 'started')
    df = pd.read_csv('./db/train/'+file)

    if i==5:
        del df['Unnamed: 2']
        del df['Unnamed: 3']
        del df['Legal Document']

    if i==8:
        df.loc[:, 'category'] = 'Scientific Research Paper'

    # df.rename(columns={'Content': 'content',
    #           'Category': 'category'}, inplace=True)
    if file == 'Business Documents.csv':
        df.loc[:, 'category'] = 'Business Documents'

    new_df = preprocesse_df(df)

    transformed_df = new_df[['transformed_content', 'category']]
    print(transformed_df.shape)

    # message_train, message_test = train_test_split(transformed_df)

    messages_train = transformed_df.to_records(index=False)
    # messages_test = message_test.to_records(index=False)

    metric = metrics.Accuracy()
    # Training the model row by row
    for content, category in messages_train:
        model = model.fit_one(content, category)
        y_pred = model.predict_one(content)
        metric = metric.update(category, y_pred)
        true_labels.append(category)
        predicted_labels.append(y_pred)
    print(metric)

    # # test Data Accuracy
    # test_metric = metrics.Accuracy()
    # for content, category in messages_test:
    #     y_pred = model.predict_one(content)
    #     test_metric = metric.update(category, y_pred)
    # print(test_metric)

    print(i, file, 'successful ------------------------------')

0 Business Documents.csv started
(413, 2)
Accuracy: 100.00%
0 Business Documents.csv successful ------------------------------
1 Creative Documents.csv started
(0, 2)
Accuracy: 0.00%
1 Creative Documents.csv successful ------------------------------
2 Educational Documents.csv started
(14, 2)
Accuracy: 100.00%
2 Educational Documents.csv successful ------------------------------
3 Financial Documents.csv started
(47, 2)
Accuracy: 100.00%
3 Financial Documents.csv successful ------------------------------
4 Govt Documents.csv started


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x96 in position 3418: invalid start byte

In [13]:
with open('predict-doc3.pkl', 'wb') as f:
    pickle.dump(model, f)